# Model Iteration 1
Here, I want to implement a very simple model. To do this, I'm not going to create very many features, just recode some of the variables that are categorical like district. 

## Importing Everything!

In [1]:
% matplotlib inline
import helperFunctions

import shapefile
import pandas as pd
import numpy as np
import itertools

import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib import cm
from datetime import datetime
from ipywidgets import widgets  
from IPython.display import display

from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import KFold
from sklearn.cross_validation import train_test_split
from sklearn.datasets import load_iris
from sklearn import tree
from sklearn.metrics import log_loss


## Loading in the data
Now, I need to load in the data

In [2]:
crimeData = pd.read_csv('train.csv')

## Recoding Columns
Here, I want to do some recoding of the columns. To do this, I'm going to use our handy-dandy helper function.  

In [3]:
crimeData, addedColumns = helperFunctions.recodeData(
    crimeData, isTrain = True)
crimeData

,Dates,Category,DayOfWeek,PdDistrict,Address,X,Y,DateTime,Year,Month,...,DistrictNORTHERN,DistrictPARK,DistrictINGLESIDE,DistrictBAYVIEW,DistrictRICHMOND,DistrictCENTRAL,DistrictTARAVAL,DistrictTENDERLOIN,DistrictMISSION,DistrictSOUTHERN
0,2015-05-13 23:53:00,WARRANTS,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015-05-13 23:53:00,2015,5,...,1,0,0,0,0,0,0,0,0,0
1,2015-05-13 23:53:00,OTHER OFFENSES,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015-05-13 23:53:00,2015,5,...,1,0,0,0,0,0,0,0,0,0
2,2015-05-13 23:33:00,OTHER OFFENSES,Wednesday,NORTHERN,VANNESS AV / GREENWICH ST,-122.424363,37.800414,2015-05-13 23:33:00,2015,5,...,1,0,0,0,0,0,0,0,0,0
3,2015-05-13 23:30:00,LARCENY/THEFT,Wednesday,NORTHERN,1500 Block of LOMBARD ST,-122.426995,37.800873,2015-05-13 23:30:00,2015,5,...,1,0,0,0,0,0,0,0,0,0
4,2015-05-13 23:30:00,LARCENY/THEFT,Wednesday,PARK,100 Block of BRODERICK ST,-122.438738,37.771541,2015-05-13 23:30:00,2015,5,...,0,1,0,0,0,0,0,0,0,0
5,2015-05-13 23:30:00,LARCENY/THEFT,Wednesday,INGLESIDE,0 Block of TEDDY AV,-122.403252,37.713431,2015-05-13 23:30:00,2015,5,...,0,0,1,0,0,0,0,0,0,0
6,2015-05-13 23:30:00,VEHICLE THEFT,Wednesday,INGLESIDE,AVALON AV / PERU AV,-122.423327,37.725138,2015-05-13 23:30:00,2015,5,...,0,0,1,0,0,0,0,0,0,0
7,2015-05-13 23:30:00,VEHICLE THEFT,Wednesday,BAYVIEW,KIRKWOOD AV / DONAHUE ST,-122.371274,37.727564,2015-05-13 23:30:00,2015,5,...,0,0,0,1,0,0,0,0,0,0
8,2015-05-13 23:00:00,LARCENY/THEFT,Wednesday,RICHMOND,600 Block of 47TH AV,-122.508194,37.776601,2015-05-13 23:00:00,2015,5,...,0,0,0,0,1,0,0,0,0,0
9,2015-05-13 23:00:00,LARCENY/THEFT,Wednesday,CENTRAL,JEFFERSON ST / LEAVENWORTH ST,-122.419088,37.807802,2015-05-13 23:00:00,2015,5,...,0,0,0,0,0,1,0,0,0,0


## Model Iteration 1
Now that I've done some recoding, I'm going to create my model. To do this, I'm going to do a random forest classifier. 

In [4]:
columnsToUse = addedColumns

X = crimeData[columnsToUse]
y = crimeData['Category']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=42)

clf = RandomForestClassifier(n_estimators=10, max_depth=None, min_samples_split=1, random_state=0)
clf.fit(X_train, y_train)
probs = clf.predict_proba(X_test)

score = log_loss(y_test, probs)
print(score)
# clf.score(X_test, y_test)  

18.207860442


This submission function came from [this script](https://www.kaggle.com/shifanmao/sf-crime/random-forest-2/code)

We modified this slightly!

In [7]:
def make_submission(clf, path='my_submission.csv'):
    '''This function will take in a model and a an optional 
    filepath and create a submissision file for us.'''
   
    test_data = pd.read_csv('test.csv')
    
    test_data, columnsToUse = helperFunctions.recodeData(test_data)
    
    X_test_data = test_data[columnsToUse]
    ids = test_data['Id']
    
    #Predict the probabilities of each crime. 
    y_prob = clf.predict_proba(X_test)

    with open(path, 'w') as f:
        f.write("ID,")
        f.write(','.join(clf.classes_))
        f.write('\n')

        for id, probs in zip(ids, y_prob):
#            probas = ','.join([id] + list(map(str, probs.tolist())))
            probas = ','.join([str(id)] + map(str,["%.2f" % elem for elem in list(map(float,probs.tolist()))]))
#            probas = ','.join(list(map(str, probs.tolist())))
            f.write(probas)
            f.write('\n')
    print(" -- Wrote submission to file {}.".format(path))


In [8]:
make_submission(clf)

 -- Wrote submission to file my_submission.csv.
